In [1]:
from pathlib import Path
from datetime import timedelta
from utinni import Context
from tiro.core import Scenario
from tiro.plugins.utinni import TiroTSPump

from rich import print

In [2]:
scenario = Scenario.from_yaml(Path("scenario/scenario.yaml"),
                              Path("scenario/use-srv1.yaml"), Path("scenario/use-srv2.yaml"))
pump = TiroTSPump(
    scenario=scenario,
    influxdb_url="http://localhost:8086",
    influxdb_token="tiro-token",
    influxdb_org="tiro",
    arangodb_db = "tiro",
    arangodb_graph = "scenario",
    arangodb_hosts = "http://localhost:8529",
    arangodb_auth=dict(password="tiro-password")
).bind(
    influxdb_bucket="tiro",
    influxdb_measurement="tiro_telemetry"
)

In [3]:
context = Context()
context.add_pump("tiro", pump)

In [4]:
context.bind(
    start=-timedelta(hours=1),
    step=timedelta(minutes=5)
)

In [29]:
print(
context.tiro_table(
    "%%Server%FlowRate", 
    type="historian",
    column="Rack", 
    asset_agg_fn="sum"
)["FlowRate"].value
     )

rack_0   rack_1
2022-09-27 15:10:00+08:00  2314.28  1595.97
2022-09-27 15:15:00+08:00  1684.89   725.40
2022-09-27 15:20:00+08:00  1704.01  1146.60
2022-09-27 15:25:00+08:00  2469.59   888.24
2022-09-27 15:30:00+08:00  1484.55  1831.56
2022-09-27 15:35:00+08:00  1395.31  1706.43
2022-09-27 15:40:00+08:00  1489.70  1741.37
2022-09-27 15:45:00+08:00  1842.68  1008.99
2022-09-27 15:50:00+08:00  1593.35  1911.79
2022-09-27 15:55:00+08:00  1888.52   999.81
2022-09-27 16:00:00+08:00  1587.14  2006.45
2022-09-27 16:05:00+08:00  1752.09  1323.92
2022-09-27 16:10:00+08:00  1629.37  1596.03

In [32]:
print(context.tiro_table(
    "%%ActivePower",
    type="status", 
    # as_dataframe=False, 
    value_only=True
).value)

{
    'DataHall': {
        'data_hall_0': {
            'CRAC': {'crac_0': {'Telemetry': {'ActivePower': 57.83}}},
            'Rack': {
                'rack_0': {'Telemetry': {'ActivePower': 955.42}},
                'rack_1': {'Telemetry': {'ActivePower': 600.6}}
            }
        }
    }
}

In [52]:
print(
context.tiro_table(
    "%%FlowRate",
    type="status", 
    as_dataframe=True,
    include_tags=["Rack", "Server"],
    value_only=True,
)["FlowRate"].value
)

Rack    Server   value
0  rack_0  server_0  876.44
1  rack_0  server_1  443.27
2  rack_0  server_2  369.87
3  rack_1  server_3  535.70
4  rack_1  server_4  476.11
5  rack_1  server_5  160.21